In [ ]:
pip install pandas openpyxl -q

In [ ]:
!pip install gurobipy -q

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Create an environment with your WLS license
params = {
"WLSACCESSID": '8aa21872-829c-4114-b577-3e764649bd8e',
"WLSSECRET": '0cd1bef5-83d6-4f09-8d59-1b559995ac13',
"LICENSEID": 964132,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(env=env)
from gurobipy import quicksum

In [ ]:
import pandas as pd
# Read data from Excel files
price_data = pd.read_excel("/kaggle/input/capstone/price_data.xlsx")
demand_data = pd.read_excel("/kaggle/input/capstone/demand_data.xlsx", index_col=0)
threshold_data = pd.read_excel("/kaggle/input/capstone/thresholds.xlsx")
spec_threshold_data = pd.read_excel("/kaggle/input/capstone/spec_threshold.xlsx")
supplier_groups_data = pd.read_excel("/kaggle/input/capstone/supplier_groups.xlsx")

# Create dictionaries for data
D = {(i, m): demand_data.at[i, m] for i in demand_data.index for m in demand_data.columns}
C = {(row["Material"], row["Supplier"], row["Location"], row["Container"], row["Plant"], row["Region"]): row["Price"] for _, row in price_data.iterrows()}
V = {(row["Supplier"]): row["Threshold"] for _, row in threshold_data.iterrows()}
T = {(row["Supplier"]): row["Spect"] for _, row in spec_threshold_data.iterrows()}
G = {group: supplier_groups_data[supplier_groups_data["Groups"]==group]["Suppliers"].unique().tolist() for group in supplier_groups_data["Groups"].unique()}


# Rest of the sets
I = demand_data.index.unique()
J = price_data["Supplier"].unique()
K = price_data["Location"].unique()
L = price_data["Container"].unique()
M = demand_data.columns
H = G.keys()
OLV = {"Orange", "Lime", "Olive"}
N = price_data["Region"].unique()

# Calculate the number of suppliers for each material without considering suppliers in the same group,
# and only counting suppliers that are able to serve both plant "TPVA" and plant "TPNM" for the same chemical
suppliers_for_material = {}
for i in I:
    suppliers_set = set()
    suppliers_tpva = set(j for (i_, j, k, l, m, n) in C.keys() if i == i_ and m == "TPVA")
    suppliers_tpnm = set(j for (i_, j, k, l, m, n) in C.keys() if i == i_ and m == "TPNM")
    
    common_suppliers = suppliers_tpva.intersection(suppliers_tpnm)
    suppliers_set.update(common_suppliers)

# Remove duplicate suppliers from the same group
    for h in H:
        group_suppliers = set(G[h])
        intersection = suppliers_set.intersection(group_suppliers)
        if len(intersection) > 1:
            suppliers_set -= intersection
            suppliers_set.add(next(iter(intersection)))
    suppliers_for_material[i] = len(suppliers_set)


# Calculate the number of unique regions for each material
regions_for_material = {}
for i in I:
    regions_set = set()
    for (i_, j, k, l, m, n) in C.keys():
        if i == i_:
            regions_set.add(n)
    regions_for_material[i] = len(regions_set)


In [ ]:
### Create variables ###
z = model.addVars(J, I, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="z") #variable for maximum share of supplier up to 51%
w = model.addVars(J, I, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="w") #variable for maximum share of supplier up to 33.34%
r = model.addVars(I, N, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="r") #variable for maximum share of region
x = model.addVars(C.keys(), vtype=GRB.INTEGER, name="x") #Volume of material supplied
y = {} #binary variable if a supplier j from group G[h] is supplying (active)
for h in H:
    for j in G[h]:
        y[j, h] = model.addVar(vtype=GRB.BINARY, name=f"y_{j}_{h}")

# Set default constraints limiting max share of single supplier & regions
default_max_percentage1 = 1
default_max_percentage2 = 1
default_max_region_share = 0.50001
        
for i in I:
    for n in N:
        r[i, n].setAttr(GRB.Attr.UB, default_max_region_share)

for j in J:
    for i in I:
        z[j, i].setAttr(GRB.Attr.UB, default_max_percentage1)

for j in J:
    for i in I:
        w[j, i].setAttr(GRB.Attr.UB, default_max_percentage2)
        
### Set objective function. Minimize total cost = Cost of material Cijklm * Volume of material Xijklm
model.setObjective(sum(C[i, j, k, l, m, n] * x[i, j, k, l, m, n] for (i, j, k, l, m, n) in C.keys()), GRB.MINIMIZE)


### Add constraints ###

#For each material i and plant m, the sum of the supplied volume x, from active suppliers (y=1), has to be >= to demand D for material i at plant m
for i in I:
    for m in M:
        model.addConstr(
            sum(x[i, j, k, l, m, n] * y.get((j, h), 0) for h in H for (i_, j, k, l, m_, n) in C.keys() if i == i_ and m == m_ and (j, h) in y) +
            sum(x[i, j, k, l, m, n] for (i_, j, k, l, m_, n) in C.keys() if i == i_ and m == m_ and all((j, h) not in y for h in H)) >= D[i, m], f"Demand_{i}_{m}")

                
# For each group h, there can be at most 1 active supplier
for h in H:
    model.addConstr(sum(y[j, h] for j in G[h]) <= 1, f"One_Supplier_Per_Group_{h}")

# Add new constraint for maximum volume supplied by a single supplier for each material
for i in I:
    for j in J:
        if suppliers_for_material[i] > 1:
            model.addConstr(
                sum(x[i, j, k, l, m, n] for (i_, j_, k, l, m, n) in C.keys() if i == i_ and j == j_) <= z[j, i] * sum(D[i, m] for m in M),f"Max_Volume_{i}_{j}")
    
# for i in I:
    for j in J:
        if suppliers_for_material[i] > 2:
            model.addConstr(
                sum(x[i, j, k, l, m, n] for (i_, j_, k, l, m, n) in C.keys() if i == i_ and j == j_) <= w[j, i] * sum(D[i, m] for m in M),f"Max_Volume_{i}_{j}")
     

                
# Add constraint for maximum share of a single region if a material is available from more than 1 region
for i in I:
    for n in N:
        if regions_for_material[i] > 1:
            constr_name = f"Max_Region_Share_{i}_{n}"
            model.addConstr(
                sum(x[i, j, k, l, m, n_] for (i_, j, k, l, m, n_) in C.keys() if i == i_ and n == n_) <= r[i, n] * sum(D[i, m] for m in M),
                constr_name)

  
                
#For supplier j in group h, if they are active, they have to supply a minimum quantity V[j]
for h in H:
    for j in G[h]:
        if j in V:
            model.addConstr(
                sum(x[i, j, k, l, m, n] for (i, j_, k, l, m, n) in C.keys() if j == j_) >= V[j] * y[(j, h)],
                f"Min_V_{j}")
            

#If supplier j has a specific threshold t, then the sum of amount supplied from chemicals OLV, has to be >= threshold T for supplier j
for h in H:
    for j in T:
        if T[j] > 0:
            model.addConstr(
                sum(x[i, j, k, l, m, n] for (i, j_, k, l, m, n) in C.keys() if i in OLV and j == j_) >= T[j] * y.get((j,h),0),
                f"MinOLV_{j}")

 
    
# Non-negativity is automatically enforceyd by Gurobi for the decision variables

model.setParam("Presolve", 2)
model.setParam("MIPGap", 0.00005)  # 0.005% optimality gap
model.setParam("TimeLimit", 120)  # 2 minutes timeout limit



# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal cost: {model.objVal}")
    total_volume_supplied = 0
    for (i, j, k, l, m, n), var in x.items():
        total_volume_supplied += var.x
        
    print(f"Total volume supplied: {total_volume_supplied}")

In [ ]:
# Check which of the supplier discount level is used
y

In [ ]:
# Extract results
result_data = [(i, j, k, l, m, n, x[i, j, k, l, m, n].x) for (i, j, k, l, m, n) in C.keys() if x[i, j, k, l, m, n].x > 0]

# Create DataFrame
result = pd.DataFrame(result_data, columns=["Material", "Supplier", "Location", "Container", "Plant", "Region", "Quantity"])

# Calculate cost for each row
result["Cost"] = result.apply(lambda row: C[row["Material"], row["Supplier"], row["Location"], row["Container"], row["Plant"], row["Region"]] * row["Quantity"], axis=1)

In [ ]:
# Export result DataFrame to an Excel file
result.to_excel("optimization_result3sup.xlsx", index=False)

In [ ]:
result